In [42]:
import glob
import pandas as pd

csv_files = glob.glob("data/boxscores/game-info/*.csv")
combined_df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

team_map = pd.read_csv("data/teams/map.csv")[["team", "espn"]]

In [43]:
combined_df = combined_df.merge(team_map, left_on="home_team", right_on="espn", how="left").merge(team_map, left_on="away_team", right_on="espn", how="left")

combined_df['home'] = combined_df['team_x']
combined_df['away'] = combined_df['team_y']

combined_df = combined_df.dropna(subset="home").dropna(subset="away").dropna(subset="home_1h")
combined_df = combined_df[['game_id', 'date_utc', 'time_utc', 'neutral_site', 'home',
       'away', 'home_1h', 'away_1h', 'home_2h', 'away_2h', 'home_score',
       'away_score']]

In [46]:
import glob
import pandas as pd

csv_files = glob.glob("data/boxscores/team-stats/*.csv")
team_combined_df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

In [45]:
combined_df.sort_values("date_utc")

,game_id,date_utc,time_utc,neutral_site,home,away,home_1h,away_1h,home_2h,away_2h,home_score,away_score
193,401259878,2020-11-25,19:00Z,False,South Carolina St.,Appalachian St.,20.0,47.0,41.0,34.0,61.0,81.0
3090,401260050,2020-11-25,23:00Z,False,Butler,Western Michigan,32.0,26.0,34.0,36.0,66.0,62.0
19411,401262371,2020-11-25,19:00Z,False,Illinois,North Carolina A&T,58.0,19.0,64.0,41.0,122.0,60.0
19770,401259897,2020-11-25,23:00Z,True,Purdue,Liberty,39.0,32.0,38.0,32.0,77.0,64.0
9224,401263443,2020-11-25,21:00Z,False,Iowa,North Carolina Central,44.0,29.0,53.0,38.0,97.0,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6568,401744252,2025-03-14,01:00Z,True,Abilene Christian,Seattle,25.0,30.0,38.0,39.0,63.0,69.0
7467,401744013,2025-03-14,00:30Z,True,Marist,Mount St. Mary,19.0,27.0,39.0,35.0,58.0,62.0
10372,401743881,2025-03-14,02:18Z,True,Connecticut,Villanova,31.0,36.0,42.0,20.0,73.0,56.0
23973,401744550,2025-03-14,01:00Z,True,Middle Tennessee,Louisiana Tech,40.0,31.0,22.0,31.0,77.0,75.0


In [66]:
df = team_combined_df.sort_values(['displayOrder'])

# create a helper column to pair home/away rows by game id
# (if you don't already have a game_id column)
# Split into home and away
home_df = df[df['homeAway'] == 'home'].copy()
away_df = df[df['homeAway'] == 'away'].copy()

# Columns we don't want duplicated (they’ll be renamed anyway)
cols_to_remove = ['homeAway', 'displayOrder', 'abbreviation', 'team_id']

# Rename columns to indicate home/away
home_df = home_df.drop(columns=cols_to_remove).add_suffix('_home')
away_df = away_df.drop(columns=cols_to_remove).add_suffix('_away')

# Merge back together on the shared game_id
# (keep original game_id)
final_df = pd.merge(
    home_df,
    away_df,
    left_on='game_id_home',
    right_on='game_id_away',
    suffixes=('', ''),
    how='inner'
)

# Keep just one copy of game_id
final_df['game_id'] = final_df['game_id_home']
final_df = final_df.drop(columns=['game_id_home', 'game_id_away'])

# Optional: reorder columns to have game_id first
cols = ['game_id'] + [c for c in final_df.columns if c != 'game_id']
final_df = final_df[cols]

In [67]:
final_df = final_df[['game_id', 'assists_home', 'defensiveRebounds_home', 'freeThrowPct_home',
       'threePointFieldGoalsMade-threePointFieldGoalsAttempted_home',
       'fouls_home', 'totalRebounds_home', 'threePointFieldGoalPct_home',
       'teamTurnovers_home', 'pointsInPaint_home', 'technicalFouls_home',
       'totalTechnicalFouls_home', 'largestLead_home',
       'offensiveRebounds_home', 'fieldGoalPct_home',
       'totalTurnovers_home', 'turnoverPoints_home', 'flagrantFouls_home',
       'freeThrowsMade-freeThrowsAttempted_home', 'steals_home',
       'fieldGoalsMade-fieldGoalsAttempted_home', 'blocks_home',
       'fastBreakPoints_home', 'turnovers_home',  'assists_away',
       'defensiveRebounds_away', 'freeThrowPct_away',
       'threePointFieldGoalsMade-threePointFieldGoalsAttempted_away',
       'fouls_away', 'totalRebounds_away', 'threePointFieldGoalPct_away',
       'teamTurnovers_away', 'pointsInPaint_away', 'technicalFouls_away',
       'totalTechnicalFouls_away', 'largestLead_away',
       'offensiveRebounds_away', 'fieldGoalPct_away',
       'totalTurnovers_away', 'turnoverPoints_away', 'flagrantFouls_away',
       'freeThrowsMade-freeThrowsAttempted_away', 'steals_away',
       'fieldGoalsMade-fieldGoalsAttempted_away', 'blocks_away',
       'fastBreakPoints_away', 'turnovers_away']]

In [68]:
final_df

,game_id,assists_home,defensiveRebounds_home,freeThrowPct_home,threePointFieldGoalsMade-threePointFieldGoalsAttempted_home,fouls_home,totalRebounds_home,threePointFieldGoalPct_home,teamTurnovers_home,pointsInPaint_home,...,fieldGoalPct_away,totalTurnovers_away,turnoverPoints_away,flagrantFouls_away,freeThrowsMade-freeThrowsAttempted_away,steals_away,fieldGoalsMade-fieldGoalsAttempted_away,blocks_away,fastBreakPoints_away,turnovers_away
0,401487711,15.0,22.0,61.9,9-16,19.0,25.0,56.3,0.0,26.0,...,53.6,15.0,18.0,0.0,7-17,5.0,30-56,2.0,3.0,15.0
1,401384886,8.0,23.0,65.0,5-14,24.0,37.0,35.7,0.0,0.0,...,50.0,13.0,16.0,0.0,18-21,7.0,26-52,3.0,6.0,13.0
2,401490224,12.0,21.0,63.6,8-27,15.0,33.0,29.6,0.0,28.0,...,48.1,9.0,6.0,0.0,12-16,4.0,25-52,2.0,2.0,9.0
3,401369249,11.0,28.0,69.2,5-20,16.0,35.0,25.0,1.0,46.0,...,40.0,18.0,8.0,0.0,11-17,9.0,28-70,0.0,6.0,18.0
4,401601271,17.0,22.0,78.9,13-28,16.0,33.0,46.4,0.0,26.0,...,51.9,11.0,14.0,0.0,13-18,4.0,27-52,3.0,0.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27451,401370964,10.0,31.0,66.7,5-19,10.0,48.0,26.3,0.0,38.0,...,38.2,9.0,4.0,0.0,8-11,4.0,21-55,4.0,5.0,9.0
27452,401473541,12.0,19.0,50.0,9-23,11.0,26.0,39.1,0.0,28.0,...,59.7,15.0,19.0,0.0,6-7,7.0,37-62,4.0,11.0,15.0
27453,401411700,14.0,23.0,92.9,4-20,13.0,36.0,20.0,1.0,0.0,...,38.8,16.0,0.0,0.0,5-9,6.0,26-67,2.0,0.0,16.0
27454,401625451,10.0,27.0,73.0,5-21,22.0,48.0,23.8,0.0,36.0,...,41.7,12.0,5.0,0.0,12-21,4.0,25-60,7.0,22.0,12.0


In [71]:
combined_df = combined_df.merge(final_df, on="game_id")

In [72]:
import glob
import pandas as pd

csv_files = glob.glob("data/boxscores/oficials/*.csv")
officials_df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

In [74]:
# create a rank/order number per game_id
officials_df['official_number'] = officials_df.groupby('game_id').cumcount() + 1

# pivot to wide format
flat_officials = (
    officials_df.pivot(index='game_id', columns='official_number', values='official_name')
    .rename(columns=lambda x: f'official_{x}')
    .reset_index()
)

/Users/nickdimmitt/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [76]:
flat_officials = flat_officials[["game_id", "official_1", "official_2", "official_3"]]

In [79]:
combined_df = combined_df.merge(flat_officials, on="game_id")

In [81]:
combined_df.to_csv("data/train/game-info.csv")